In [1]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import time
from concurrent.futures import ThreadPoolExecutor  
import concurrent.futures

In [2]:
def get_systemcall_name_perf(line):
    l = re.split(r' |\( |\)', line)
    l = list(filter(lambda a: a != '', l))
    # print(l)
    if len(l) < 6:
        return None
    timestamp = l[0]
    time_cost = l[1]
    pid = l[4]
    if l[5] == '...':
        if timestamp == '0.000':
            return None
        else:
            syscall = l[7].split('(')[0]
    else:
        syscall = l[5].split('(')[0]
    # print(timestamp,time_cost, pid,  syscall)
    return [pid, timestamp, syscall, time_cost]

In [3]:
def change_time(df):
    index= 0
    time = 0
    for index,time in enumerate(df['timestamp']):
        if time != 0:
            break
    for j in range(0,index):
        df.loc[j, 'timestamp'] = time
    return df

In [4]:
def split_data_by_time(savePath, malware_type, pid, rounds,df,time_window):
    start = 0
    end = 0
    ti =  df['timestamp'].tolist()
    start_time = ti[0]
    end_time = 0
    index = 0
    for i in range(0, len(df)):
        end = i
        end_time=ti[end]
        if end_time - start_time >= time_window * 1000:
            fileName = '{}{}_{}_{}_{}_{}_{}.csv'.format(savePath,malware_type,pid, rounds, str(index),str(time_window),str(start_time))
            d = df[start:end]
            d.to_csv(fileName,index=None)
            index += 1
            start = end
            start_time = end_time         



In [5]:
def get_maltype_from_name(filename):
    ns = filename.split('_')
    malware_type = ''
    pid = ''
    rounds = ''
    if ns[1] != 'v2':
        malware_type =  ns[0]
        pid=  ns[1]
        rounds =  ns[2]
    else:
        malware_type = ns[0]+ns[1]
        id=  ns[2]
        rounds =  ns[3]
    return malware_type, pid, rounds

In [9]:
rootPath ='D:/git/IoT_Sensors_Security_Analysis/data/perf/'
device = 'pi3'
perfdataPath = rootPath+'{}/t1/'.format(device)
files = os.listdir(perfdataPath)

In [3]:
len(files)

43

In [ ]:
for fi in files:
    if 'txt' in fi:
        syscall_trace = []
        with open(perfdataPath + fi, 'r') as f:
            for line in f:
                res = get_systemcall_name_perf(line)
                if res != None:
                    syscall_trace.append(res)
                else: 
                    print(fi)
            f.close()
        df = pd.DataFrame(syscall_trace)
        df.columns=['pid', 'timestamp', 'syscall', 'time_cost']
        path = rootPath + '{}/data_1/'.format(device) + fi
        df.to_csv(path, index=None)
        # print(fi, set(df['syscall']))

In [15]:
dataPath = rootPath + '{}/data_1/'.format(device)

In [16]:
# tws = [10, 15, 16, 17, 20, 25, 30, 35, 40, 50, 60]
tws = [70]
datafiles = os.listdir(dataPath)
newdataPaht= rootPath+'{}/splited_1/'.format(device)
for tw in tws:
    ts = newdataPaht+ str(tw) +'/'  
    ls = os.listdir(newdataPaht)
    if str(tw) not in ls:
        os.mkdir(ts)
        ts = newdataPaht + str(tw) +'/'  
    par = tqdm.tqdm(total = len(datafiles), ncols=100)
    for fi in datafiles:
        if '.txt' in fi:
            par.update(1)
            malware_type, pid, rounds = get_maltype_from_name(fi)
            df = pd.read_csv(dataPath+fi)
            time_window = tw
            split_data_by_time(ts, malware_type, pid, rounds,df,time_window)
    par.close()


  2%|█▌                                                           | 9/360 [02:01<1:19:13, 13.54s/it]
C:\Users\luke-\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|█████████████████████████████████████████████████████████████| 360/360 [21:18<00:00,  3.55s/it]
